In [9]:
!pip install jieba

  Created wheel for jieba: filename=jieba-0.42.1-cp37-none-any.whl size=19314482 sha256=56224a34d550fdaf9e3cbe8ab4cc688eb2ebc0828d1c5d8c8d949fa96f142d16
  Stored in directory: C:\Users\WIN10\AppData\Local\pip\Cache\wheels\af\e4\8e\5fdd61a6b45032936b8f9ae2044ab33e61577950ce8e0dec29
Successfully built jieba


In [10]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import jieba
#from konlpy.tag import Okt

<br>
<br>
# 데이터 로드

Seq2Seq에서의 임베딩이 이전 예제와 다른 점은 아래와 같이 태그를 사용한다는 것입니다. 임베딩의 0~3번째에 각각 PADDING, START, END, OOV 태그를 넣습니다. 사실 그냥 똑같은 단어라고 보시면 됩니다. 다만 이 단어들이 Seq2Seq의 동작을 제어합니다. 

예를 들어, 디코더 입력에 START가 들어가면 디코딩의 시작을 의미합니다. 반대로 디코더 출력에 END가 나오면 디코딩을 종료합니다.
<br>
<br>
<br>

In [18]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
csv_path = r"C:\Users\WIN10\Desktop\chaotbotproject\chatbot\model\chinense_sample_data.xlsx"

chatbot_data = pd.read_excel(csv_path)
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

챗봇의 훈련을 위해서 송영숙님이 공개한 한글 데이터셋을 로드합니다. 질문과 대답, 감정 등 총 3개의 항목으로 구성되어 있습니다. 감정 분류는 Seq2Seq에 필요가 없기 때문에 사용하지 않습니다.

https://github.com/songys/Chatbot_data
<br>
<br>
<br>

In [12]:
# 데이터 개수
len(question)


11823

In [19]:
# 데이터의 일부만 학습에 사용
question = question[:100]
answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 12点  错！
A : 一天 又 过去 了

Q : 第一 志愿 学校 落榜 了
A : 慰劳 您

Q : 想 去 玩 4天3夜
A : 旅行 什么时候 都 好

Q : 想 出去 玩 四天三夜
A : 旅行 什么时候 都 好

Q : PPL 太 过分 了
A : 皱起 眉头 吧

Q : SD卡 坏 了
A : 重新 买 新 更 放心 了

Q : SD卡 不行
A : 重新 买 新 更 放心 了

Q : 为什么 不 玩 SNS 双臂 呢    
A : 也有 可能 不知道

Q : 我 知道 SNS 浪费时间   但是 每天 都 在 做
A : 定下 时间 再 做 吧

Q : SNS 是 浪费 时间 ， 但是 总 想 看
A : 定下 时间 再 做 吧



<br>
<br>
# 단어 사전 생성

In [20]:
# 형태소분석 함수
def pos_tag(sentences):
    #중국어 형태소 분석기 설정
    #j.enable_paddle()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        #sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        print(sentences)
        
    return sentences_pos

문장을 먼저 최소 단위인 토큰으로 나누어야 합니다. 한글 문장이라면 보통 KoNLPy(코엔엘파이)를 통해 형태소분석을 적용합니다. Hannanum, Kkma, Komoran, Mecab, Okt(Twitter) 등 여러가지 라이브러리를 동일한 인터페이스로 실행이 가능합니다.

KoNLPy는 자바로 구현이 되어 있습니다. 맥OS와 달리 윈도우 아나콘다는 아래와 같은 작업을 거쳐야 합니다.

< 1. 자바 설치 ><br>
커맨드창에서 'java -version' 명령어를 실행하여 버전이 1.7 이상이어야 합니다.

< 2. JPype1 설치 ><br>
파이썬에서 자바를 호출할 수 있는 라이브러리입니다.

< 3. KoNLPy 설치 ><br>
위의 두 가지가 미리 완료되어 있어야만 가능합니다.

아래 링크에서 좀 더 자세한 방법을 확인하시기 바랍니다.<br>
-> https://ericnjennifer.github.io/python_visualization/2018/01/21/PythonVisualization_Chapt1.html
<br>
<br>
<br>

In [22]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


会 吧', '我 也是', '看来 大家 都 很 忙 没有 时间 聊天 了', '看来 大家 都 很 忙 没有 时间 聊天 了', '全家 都 去 称 心 如意 的 地方 吧', '好 主意 。', '应该 会 成为 变得 更 亲近 的 机会 吧', '把 创造 我的人 当做 父母 ， 和 我 聊天 的 人 当作 朋友 。', '好 主意 。', '真 的 是 会 后悔 的 习惯', '不要 做出 鲁莽 的 决定', '请 联系 老师 或 机关', '紧张 的 感情 本身 就 很 珍贵', '希望 能得到 很多 东西', '也 需要 休息', '买 两个 甜 咸 的 才是 真理', '买 两个 甜 咸 的 才是 真理', '请 慢 用', '我 也 不 喜欢', '去 吧', '去 吧', '请 慢 用', '请 慢 用', '去 医院 吧', '这时候 好好 休息 很重要', '这时候 好好 休息 很重要', '这时候 好好 休息 很重要', '温暖 地 管理 吧', '去 医院 吧', '去 医院 吧', '我 也想听', '请 更爱 自己', '那 是 习惯', '那 是 习惯', '你 掐 着 问 吧', '只 想着 好事情 吧', '心痛 吗', '好像 很 突然', '看来 关系 有 了 变化 了', '前3秒 很重要   展示 一下 你 的 魅力 吧', '能 承担 的 时候 再 养 吧', '先 看看 生活方式', '先 看看 生活方式', '能 承担 的 时候 再 养 吧', '是 美丽 的 地方', '也 没 什么 不 可以 的', '一个人 也 好', '恋人 就算 胖 了 也 看 不 出来 ， 就算 看 出来 也 不会 讨厌 的', '祝 你 玩 得 愉快 ！', '别 拖拖拉拉 的', '说说 看', '在一起 的 话 会 更 了解 彼此 的', '开始 吧', '开始 吧', '快 放假 了', '假期 很 短 吧', '摆脱 出来 比较好', '摆脱 出来 比较好', '洗 完 脸 再来 吧', '那 是 最 重要 的', '那 是 最 重要 的', '下次 开始 了解 更多 吧', '好像 很 突然', '从 公事 做起', '从 公事 做起', '午觉 可以 睡 一会儿', '也 请 喜欢 我', '看来 是 想念 朋友们 了', 

In [25]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))
print(len(words))
# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]
print(words)

377
['<PADDING>', '<START>', '<END>', '<OOV>', '相信', '谁', '开', '朋友们', '乞丐', '心痛', '出去', '快', '困', '关机', '煤气灯', '零食', '但', '呢', '什么', '敢', '回家', '要不要', '做', '悲哀', '把', '您', '留下', '忙', '除了', '一', '洗', '旁边', '开始', '好奇', '柿子干', '还是', '真好', '又', '离家出走', '最', '汗', '虚拟', '眉头', '怎么样', '希望', '煤气费', '看', '垃圾桶', '讲装', '他', '感冒', '狗屁', '不知道', '但是', '江原道', '人', '人们', 'SNS', '贵', '应该', '哪里', '公事', '放下来', '不要', '享受', '为了', '度过', '给', '同班', '概念', '很重要', '挺好', '从', '我', '紧张', '。', '志愿', '孩子', '鲁莽', '无视', '要', '两个', '不是', '或', '好事情', '游泳场', '想着', '我的人', '做出', '真理', '更', '做起', '机关', '后悔', '重新开始', '没', '好好', '总', '的', '重新', '省点', '玩', '开学', '吸烟', '喜欢', '好像', '家', '恋人', '变得', '也是', '这时候', '感冒药', '玩游戏', '趣味', '抓住', '尽量', '成为', '有', '后', '12点', '错！', '苦恼', '走', '乱七八糟', '征兆', '竟', '买', '个人', '心', '不愉快', '炫耀', '下个月', '时候', '愉快', '变成', '工作', '美丽', '重要', '慢', '联系', '告诉我', '家人们', '咸', '别', '运气', '感觉', '都', '在一起', '东西', '不行', '购物', '好久', '累', '放', '了解', '解压', '！', '是不是', '下次', '得', '全家', '真好听', '新', '休息', '就', '吃'

질문과 대답 문장들을 합쳐서 전체 단어 사전을 만듭니다. 자연어처리에서는 항상 이렇게 단어를 인덱스에 따라 정리를 해야 합니다. 그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다. 또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요합니다.
<br>
<br>
<br>

In [26]:
# 단어 개수
len(words)


381

In [ ]:
# 단어 출력
words[:20]


['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '可能',
 '自己',
 '生活方式',
 '下次',
 '老师',
 '这么',
 '最',
 '称',
 '变化',
 '好像',
 '一会儿',
 '很多',
 '家人们',
 '私事',
 '地',
 '新']

In [27]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

print(index_to_word)

{0: '<PADDING>', 1: '<START>', 2: '<END>', 3: '<OOV>', 4: '相信', 5: '谁', 6: '开', 7: '朋友们', 8: '乞丐', 9: '心痛', 10: '出去', 11: '快', 12: '困', 13: '关机', 14: '煤气灯', 15: '零食', 16: '但', 17: '呢', 18: '什么', 19: '敢', 20: '回家', 21: '要不要', 22: '做', 23: '悲哀', 24: '把', 25: '您', 26: '留下', 27: '忙', 28: '除了', 29: '一', 30: '洗', 31: '旁边', 32: '开始', 33: '好奇', 34: '柿子干', 35: '还是', 36: '真好', 37: '又', 38: '离家出走', 39: '最', 40: '汗', 41: '虚拟', 42: '眉头', 43: '怎么样', 44: '希望', 45: '煤气费', 46: '看', 47: '垃圾桶', 48: '讲装', 49: '他', 50: '感冒', 51: '狗屁', 52: '不知道', 53: '但是', 54: '江原道', 55: '人', 56: '人们', 57: 'SNS', 58: '贵', 59: '应该', 60: '哪里', 61: '公事', 62: '放下来', 63: '不要', 64: '享受', 65: '为了', 66: '度过', 67: '给', 68: '同班', 69: '概念', 70: '很重要', 71: '挺好', 72: '从', 73: '我', 74: '紧张', 75: '。', 76: '志愿', 77: '孩子', 78: '鲁莽', 79: '无视', 80: '要', 81: '两个', 82: '不是', 83: '或', 84: '好事情', 85: '游泳场', 86: '想着', 87: '我的人', 88: '做出', 89: '真理', 90: '更', 91: '做起', 92: '机关', 93: '后悔', 94: '重新开始', 95: '没', 96: '好好', 97: '总', 98: '的', 99: '重新', 10

In [28]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
dict(list(word_to_index.items())[:20])


{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '相信': 4,
 '谁': 5,
 '开': 6,
 '朋友们': 7,
 '乞丐': 8,
 '心痛': 9,
 '出去': 10,
 '快': 11,
 '困': 12,
 '关机': 13,
 '煤气灯': 14,
 '零食': 15,
 '但': 16,
 '呢': 17,
 '什么': 18,
 '敢': 19}

In [ ]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
dict(list(index_to_word.items())[:20])


{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '可能',
 5: '自己',
 6: '生活方式',
 7: '下次',
 8: '老师',
 9: '这么',
 10: '最',
 11: '称',
 12: '变化',
 13: '好像',
 14: '一会儿',
 15: '很多',
 16: '家人们',
 17: '私事',
 18: '地',
 19: '新'}

<br>
<br>
# 전처리

In [29]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type): 
    
    sentences_index = []
    
    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []
        
        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])
        
        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
            
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]
        
        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)
        print(sentences_index)
    return np.asarray(sentences_index)

Seq2Seq에서는 학습시 다음과 같이 총 3개의 데이터가 필요합니다.

인코더 입력 : 12시 땡<br>
디코더 입력 : START 하루 가 또 가네요<br>
디코더 출력 : 하루 가 또 가네요 END

원래 Seq2Seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다. 

그러나 예측시에는 이런 방식이 불가능합니다. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다. 모델 생성 부분에서 다시 자세히 설명을 드리겠습니다.
<br>
<br>
<br>

In [30]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]


 0, 0, 0, 0, 0, 0, 0, 0, 0], [242, 305, 330, 73, 364, 291, 215, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [217, 127, 187, 372, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [169, 372, 217, 49, 177, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [283, 12, 362, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [353, 233, 215, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [102, 215, 36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[119, 120, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [343, 76, 167, 319, 215, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [210, 177, 101, 226, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [210, 10, 101, 223, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

array([119, 120,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [31]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]


 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 372, 288, 104, 73, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 304, 243, 272, 7, 215, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 311, 37, 168, 215, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 352, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 339, 363, 146, 313, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 339, 363, 146, 313, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 292, 42, 232, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 99, 126, 162, 90, 374, 215, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 99, 126, 162, 90, 374, 215, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 307, 246, 52, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

array([  1, 311,  37, 168, 215,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [33]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]


 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [372, 288, 104, 73, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [304, 243, 272, 7, 215, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[311, 37, 168, 215, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [352, 25, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [339, 363, 146, 313, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [339, 363, 146, 313, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [292, 42, 232, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [99, 126, 162, 90, 374, 215, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [99, 126, 162, 90, 374, 215, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [307, 246, 52, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

array([311,  37, 168, 215,   2,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [34]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이 되어야 합니다. 디코더의 마지막 Dense 레이어에서 softmax로 나오기 때문입니다.

<br>
<br>
# 모델 생성

In [36]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.2,
                                                recurrent_dropout=0.4,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])    

지금까지의 예제는 Sequential 방식의 모델이었습니다. 하지만 이번에는 함수형 API 모델을 사용했습니다. 인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능하기 때문입니다. 

Model() 함수로 입력과 출력을 따로 설정하여 모델을 만듭니다. 그다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다.
<br>
<br>
<br>

In [38]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)



#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용합니다. 예측 모델 인코더는 훈련 모델 인코더과 동일합니다. 그러나 예측 모델 디코더는 매번 LSTM 상태값을 입력으로 받습니다. 또한 디코더의 LSTM 상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다. 

이렇게 하는 이유는 LSTM을 딱 한번의 타임 스텝만 실행하기 때문입니다. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

<br>
<br>
# 훈련 및 테스트

In [39]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [41]:
# 에폭 반복
for epoch in range(5):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

encoder_model.save("encoder_model.h5")
decoder_model.save("decoder_model.h5")    

Total Epoch : 1
accuracy : 0.92266667
loss : 0.3705414307117462
请 什么时候 都 好 

Total Epoch : 2
accuracy : 0.96433336
loss : 0.16608950912952422
请 什么时候 都 好 

Total Epoch : 3
accuracy : 0.968
loss : 0.11058435916900634
请 什么时候 都 好 

Total Epoch : 4
accuracy : 0.9713333
loss : 0.09428036004304886
全家 什么时候 都 好 

Total Epoch : 5
accuracy : 0.97466666
loss : 0.08239608705043792
去 什么时候 都 好 



학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있습니다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용됩니다. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다.

<br>
<br>
# 문장 생성

In [42]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [44]:
# 텍스트 생성
from tensorflow.keras.models import load_model

def generate_text(input_seq):

    encoder_model = load_model("encoder_model.h5")    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    decoder_model = load_model("decoder_model.h5")
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)    
    return sentence

제일 첫 단어는 START로 시작합니다. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측을 반복합니다. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의하시기 바랍니다. END 태그가 나오면 문장 생성을 종료합니다.
<br>
<br>
<br>

In [45]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('我 爱 你')
input_seq


['我 爱 你']
[[73, 3, 278, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


array([[ 73,   3, 278,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [46]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


'也 没 什么 不 可以 的 '

데이터셋에 있는 문장과 똑같은 입력을 넣으니, 역시 정확히 일치하는 답변이 출력되었습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 같이 놀러가고 싶다')
input_seq


array([[171,   3,   3,   3,   3,   3,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'全家 都 去 称 心 如意 的 地方 吧 '

데이터셋 문장에서는 없던 '같이'라는 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.
<br>
<br>
<br>

In [ ]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('3박4일 놀러가려고')
input_seq


array([[171,   3,   3,   3,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [ ]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence


'全家 都 去 称 心 如意 的 地方 吧 '

하지만 데이터셋에 없던 '가려고'로 입력을 수정하니, 전혀 다른 문장이 출력되었습니다. 이는 우리가 데이터의 일부인 100개 문장만 학습했기 때문입니다. 데이터의 개수를 늘려서 훈련할수록 일반화 능력이 더욱 높아집니다.